In [1]:
#Useful imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, asin, acos, atan
%reload_ext autoreload
%autoreload 2
plt.style.use('latex.mplstyle')

from point_configuration import PointConfiguration

In [2]:
from plots_cti import plot_point_sets_3d, plot_matrix
## Simulation without missing data, different noise levels.
from algorithms import reconstruct_mds
from algorithms import reconstruct_srls
from algorithms import reconstruct_acd
from distributed_mds import reconstruct_dwmds
from semidefinite import reconstruct_sdp

from basics import create_noisy_edm, get_edm

N = 10
m = 9
#m = 6


n = N - m
d = 3

square_size = 5

noises = np.logspace(-1,0,20)
max_it = 100

p = PointConfiguration(N, d)

import copy
errs = [[] for noise in noises]
print(errs)
errs_dict = {
    'noise':copy.deepcopy(errs),
    'MDSalternation':copy.deepcopy(errs)  ,
    'MDSoptspace':copy.deepcopy(errs)  ,
    'ACD':copy.deepcopy(errs)  ,
    'dwMDS':copy.deepcopy(errs),
    'SDR':copy.deepcopy(errs)  ,
    'SRLS':copy.deepcopy(errs) }
print(errs_dict.items())

def localization_error(point_estimate):
    err = np.linalg.norm(p.points[:n] - point_estimate[:n])**2/n
    return err

lamda = 1000
for it in range(max_it):
    print(it)
    p.set_points('random',size=square_size)
    
    for i, noise in enumerate(noises):
        failed = []
        print(noise)
        try:
            n_edges = (N*(N-1))/2
            noisy_edm = create_noisy_edm(p.edm, noise)
            err_noise = np.linalg.norm(p.edm - noisy_edm)**2/n_edges
            errs_dict['noise'][i].append(err_noise)
        except:
            failed.append('noise')

        #weights = np.ones((N,N))
        weights = np.random.binomial(n=2,size=(N,N),p=0.2)
        weights[range(N),range(N)] = 0.0
        
        # try:
        #     x_MDS = reconstruct_mds(noisy_edm, real_points=p.points, method='geometric')
        #     err_MDS = localization_error(x_MDS)
        #     errs_dict['MDS'][i].append(err_MDS)
        # except:
        #     failed.append('MDS')
        
        try:
            x_MDS = reconstruct_mds(noisy_edm, real_points=p.points, method='geometric')
            err_MDS = localization_error(x_MDS)
            errs_dict['MDSoptspace'][i].append(err_MDS)
        except:
            failed.append('MDSoptspace')
        
        try:
            x_MDS = reconstruct_mds(noisy_edm, real_points=p.points, method='geometric')
            err_MDS = localization_error(x_MDS)
            errs_dict['MDSalternate'][i].append(err_MDS)
        except:
            failed.append('MDSalternate')

        try:
            x_ACD, fs, err_edms, err_points = reconstruct_acd(noisy_edm, weights, X_0=x_MDS.copy(), real_points=p.points)
            err_ACD = localization_error(x_ACD)
            errs_dict['ACD'][i].append(err_ACD)
        except:
            failed.append('ACD')

        try:
            x_dwMDS, costs = reconstruct_dwmds(noisy_edm, X=p.points.copy(), W=weights, n=n)
            err_dwMDS = localization_error(x_dwMDS)
            errs_dict['dwMDS'][i].append(err_dwMDS)
        except:
            failed.append('dwMDS')

        try:
            x_SDR, EDMbest = reconstruct_sdp(noisy_edm, weights, lamda, p.points)
            err_SDR = localization_error(x_SDR)
            errs_dict['SDR'][i].append(err_SDR)
        except:
            failed.append('SDR')

        try:
            x_SRLS = reconstruct_srls(noisy_edm, p.points, indices=range(n))
            err_SRLS = localization_error(x_SRLS)
            errs_dict['SRLS'][i].append(err_SRLS)
        except:
            failed.append('SRLS')
        if len(failed)>0:
            print('exception occured in',failed)
        else:
            pass
            #plot_point_sets_3d([p.points, x_MDS, x_dwMDS],['original','MDS','dwMDS'], display_lines=True)
            #print('err_MDS  ',err_MDS)
            #print('err_ACD  ',err_ACD)
            #print('err_dwMDS',err_dwMDS)
            #print('err_SDR  ',err_SDR)
            #print('err_SRLS ',err_SRLS)

        
np.save('results/log_errs_dict_n{}_m{}.npy'.format(n,m),errs_dict)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
dict_items([('noise', [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]), ('MDS', [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]), ('ACD', [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]), ('dwMDS', [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]), ('SDR', [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]), ('SRLS', [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []])])
0
0.1
0.112883789168
0.12742749857
0.143844988829
0.162377673919
0.183298071083
0.206913808111
0.233572146909
0.263665089873
0.297635144163
0.335981828628
0.379269019073
0.428133239872
0.483293023857
0.545559478117
0.615848211066
0.695192796178
0.784759970351
0.88586679041
1.0
1
0.1
0.112883789168
0.12742749857
0.143844988829
0.162377673919
0.1832980710